<a href="https://colab.research.google.com/github/lstival/mo826/blob/main/P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade P2 - Fabrício Ferreira da Silva RA: 231900 e Leandro Stival RA: 263013

##Importar os dados dos pacientes

O primeiro passo foi importar a biblioteca pandas, que é usada para manipular dados, e armazenar os arquivos que foram fornecidos que estão no formato CSV em variáveis, usando o comando read_CSV e colocando o caminho do arquivo desejado.

In [ ]:
#Ignorando alguns warnings
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
import pandas as pd
import numpy as np
#Scenario 2
encounters = pd.read_csv('https://raw.githubusercontent.com/santanche/lab2learn/master/data/synthea/scenario02/csv/encounters.csv')
patients = pd.read_csv('https://raw.githubusercontent.com/santanche/lab2learn/master/data/synthea/scenario02/csv/patients.csv')
# conditions = pd.read_csv('https://raw.githubusercontent.com/santanche/lab2learn/master/data/synthea/scenario02/csv/conditions.csv')
# procedures = pd.read_csv('https://raw.githubusercontent.com/santanche/lab2learn/master/data/synthea/scenario02/csv/procedures.csv')
print(np.shape(encounters))
print(np.shape(patients))
# print(np.shape(conditions))
# print(np.shape(procedures))

(55285, 15)
(1121, 25)


In [ ]:
encounters_death = encounters[(encounters['DESCRIPTION']=='Death Certification')]
encounters_death.drop_duplicates(subset= "Id", keep= "first", inplace=True, ignore_index=True)
# print(encounters_death)
death_variants = encounters_death['REASONDESCRIPTION'].value_counts()
# print(death_variants)
# death_variants.to_csv('causas_morte.csv')

In [ ]:
patients_death = patients.dropna(subset=['DEATHDATE'])
patients_death = patients[patients['DEATHDATE'].notna()]
patients_death.drop_duplicates(subset= "Id", keep= "first", inplace=True, ignore_index=True)
# print(patients_death)

## Fatiando a base de dados
Selecionado o top 10 doenças mais letais como foco do estudo, selecionando os pacientes que possuem ao menos um encontro com ela.

In [ ]:
#Save the name of the diases that will be considered
target_diases = death_variants[:10].index.values
target_diases

array(['Sudden Cardiac Death', 'Myocardial Infarction',
       'Chronic congestive heart failure (disorder)',
       'Acute myeloid leukemia  disease (disorder)', 'Bullet wound',
       'Cardiac Arrest', 'Pulmonary emphysema (disorder)',
       'Concussion injury of brain', 'Pneumonia', 'Stroke'], dtype=object)

In [ ]:
#Encounter where the description is one of the target diases
target_encounters = encounters[encounters['REASONDESCRIPTION'].apply(lambda x: x in target_diases)]
# target_encounters.head()
print(f"Existem {target_encounters.index.size} encontros com os diagnosticos \
selecionados.")

Existem 1493 encontros com os diagnosticos selecionados.


In [ ]:
#Pacientes com ao menos uma vez tiveram registrado com essa doença
target_patients_ids = target_encounters['PATIENT'].drop_duplicates().values
target_patients = patients[patients['Id'].apply(lambda x: x in target_patients_ids)]
print(f"Existem {target_patients.index.size} pacientes que ao menos uma vez \
tiveram a ocorrência desse diagnostico.")

Existem 235 pacientes que ao menos uma vez tiveram a ocorrência desse diagnostico.


## Dataframe com os dados de interesse do paciente

In [ ]:
#Dataframe com os dados que serão utilizados para ML
df_interest_data = pd.DataFrame()
#Inserindo o Id do paciente ao Dataframe
df_interest_data['Id'] = target_patients.Id.values

## Selecionado e tratando as variáveis de interesse dos pacientes

In [ ]:
def convert_2_datetime(column):
  return pd.to_datetime(column).dt.tz_localize(None)

In [ ]:
target_encounters['START'] = convert_2_datetime(target_encounters['START'])
target_patients['BIRTHDATE'] = convert_2_datetime(target_patients['BIRTHDATE'])

### Cálculando a idade

In [ ]:
start_encounter = convert_2_datetime(target_encounters.drop_duplicates(subset= "PATIENT", keep= "last")["START"])

In [ ]:
patient_ages = (start_encounter.values - target_patients['BIRTHDATE'].values).astype('timedelta64[Y]') / np.timedelta64(1, 'Y')

In [ ]:
#Inserindo a idade dos pacientes ao dataframe de interesse
df_interest_data['age'] = patient_ages

In [ ]:
death_dates = convert_2_datetime(target_patients['DEATHDATE'].fillna(0)).values

## Criando variável alvo

In [ ]:
days_to_target = 15

In [ ]:
df_interest_data["target"] = (start_encounter.values - death_dates).astype('timedelta64[D]') / np.timedelta64(1, 'D') < days_to_target

In [ ]:
_, deaths = np.unique(df_interest_data["target"], return_counts=True)

In [ ]:
df_interest_data['target'] = df_interest_data['target'].apply(lambda x: 1 if x is True else 0)

In [ ]:
print(f"O conjunto possui {deaths[0]} pacientes que não morreram em \
{days_to_target} dias após o evento, e {deaths[1]} mortes antes dos {days_to_target} dias.")

O conjunto possui 146 pacientes que não morreram em 15 dias após o evento, e 89 mortes antes dos 15 dias.


## Prevendo se o paciente irá morrer

In [ ]:
#Modelo de regressão logística
from sklearn.linear_model import LogisticRegression

#Criando treino e teste
from sklearn.model_selection import train_test_split

#Avalição do modelo
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

In [ ]:
train, test = train_test_split(df_interest_data, train_size=0.7, stratify=df_interest_data['target'], random_state=2022)

In [ ]:
X = train['age'].values
X = np.array(X).reshape(-1, 1)
y = train['target'].values

In [ ]:
clf = LogisticRegression(random_state=2022).fit(X, y)

In [ ]:
predicts = clf.predict(X)

In [ ]:
clf.score(X, y)

0.725609756097561

In [ ]:
tn, fp, fn, tp = confusion_matrix(y, predicts).ravel()
confusion_matrix(y, predicts)

array([[93,  9],
       [36, 26]])

In [ ]:
(tn, fp, fn, tp)

(93, 9, 36, 26)

In [ ]:
balanced_accuracy_score(y, predicts)

0.6655597722960152

## Outro classificador

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [ ]:
xgb_cl = xgb.XGBClassifier()

In [ ]:
xgb_cl.fit(X, y)

XGBClassifier()

In [ ]:
preds = xgb_cl.predict(X)

In [ ]:
accuracy_score(y, preds)

0.7804878048780488

In [ ]:
tn, fp, fn, tp = confusion_matrix(y, preds).ravel()
confusion_matrix(y, preds)

array([[95,  7],
       [29, 33]])

In [ ]:
(tn, fp, fn, tp)

(95, 7, 29, 33)

In [ ]:
balanced_accuracy_score(y, preds)

0.7318153067678684

# Conjunto de dados 01

In [ ]:
encounters = pd.read_csv('https://raw.githubusercontent.com/santanche/lab2learn/master/data/synthea/scenario01/csv/encounters.csv')
patients = pd.read_csv('https://raw.githubusercontent.com/santanche/lab2learn/master/data/synthea/scenario01/csv/patients.csv')

print(np.shape(encounters))
print(np.shape(patients))

(59481, 15)
(1174, 25)


In [ ]:
encounters_death = encounters[(encounters['DESCRIPTION']=='Death Certification')]
encounters_death.drop_duplicates(subset= "Id", keep= "first", inplace=True, ignore_index=True)
# print(encounters_death)
death_variants = encounters_death['REASONDESCRIPTION'].value_counts()
# print(death_variants)
# death_variants.to_csv('causas_morte_01.csv')

In [ ]:
patients_death = patients.dropna(subset=['DEATHDATE'])
patients_death = patients[patients['DEATHDATE'].notna()]
patients_death.drop_duplicates(subset= "Id", keep= "first", inplace=True, ignore_index=True)
# print(patients_death)

## Fatiando a base de dados
Selecionado o top 10 doenças mais letais como foco do estudo, selecionando os pacientes que possuem ao menos um encontro com ela.

In [ ]:
#Save the name of the diases that will be considered
target_diases = death_variants[:10].index.values
target_diases

array(['Acute myeloid leukemia  disease (disorder)',
       'Myocardial Infarction',
       'Chronic congestive heart failure (disorder)', 'Pneumonia',
       'Sudden Cardiac Death', 'Non-small cell lung cancer (disorder)',
       'Stroke', 'Bullet wound', 'COVID-19',
       'Chronic obstructive bronchitis (disorder)'], dtype=object)

In [ ]:
#Encounter where the description is one of the target diases
target_encounters = encounters[encounters['REASONDESCRIPTION'].apply(lambda x: x in target_diases)]
# target_encounters.head()
print(f"Existem {target_encounters.index.size} encontros com os diagnosticos \
selecionados.")

Existem 1088 encontros com os diagnosticos selecionados.


In [ ]:
#Pacientes com ao menos uma vez tiveram registrado com essa doença
target_patients_ids = target_encounters['PATIENT'].drop_duplicates().values
target_patients = patients[patients['Id'].apply(lambda x: x in target_patients_ids)]
print(f"Existem {target_patients.index.size} pacientes que ao menos uma vez \
tiveram a ocorrência desse diagnostico.")

Existem 287 pacientes que ao menos uma vez tiveram a ocorrência desse diagnostico.


## Dataframe com os dados de interesse do paciente

In [ ]:
#Dataframe com os dados que serão utilizados para ML
df_interest_data = pd.DataFrame()
#Inserindo o Id do paciente ao Dataframe
df_interest_data['Id'] = target_patients.Id.values

## Selecionado e tratando as variáveis de interesse dos pacientes

In [ ]:
def convert_2_datetime(column):
  return pd.to_datetime(column).dt.tz_localize(None)

In [ ]:
target_encounters['START'] = convert_2_datetime(target_encounters['START'])
target_patients['BIRTHDATE'] = convert_2_datetime(target_patients['BIRTHDATE'])

### Cálculando a idade

In [ ]:
start_encounter = convert_2_datetime(target_encounters.drop_duplicates(subset= "PATIENT", keep= "last")["START"])

In [ ]:
patient_ages = (start_encounter.values - target_patients['BIRTHDATE'].values).astype('timedelta64[Y]') / np.timedelta64(1, 'Y')

In [ ]:
#Inserindo a idade dos pacientes ao dataframe de interesse
df_interest_data['age'] = patient_ages

In [ ]:
death_dates = convert_2_datetime(target_patients['DEATHDATE'].fillna(0)).values

## Criando variável alvo

In [ ]:
days_to_target = 15

In [ ]:
df_interest_data["target"] = (start_encounter.values - death_dates).astype('timedelta64[D]') / np.timedelta64(1, 'D') < days_to_target

In [ ]:
_, deaths = np.unique(df_interest_data["target"], return_counts=True)

In [ ]:
df_interest_data['target'] = df_interest_data['target'].apply(lambda x: 1 if x is True else 0)

In [ ]:
print(f"O conjunto possui {deaths[0]} pacientes que não morreram em \
{days_to_target} dias após o evento, e {deaths[1]} mortes antes dos {days_to_target} dias.")

O conjunto possui 181 pacientes que não morreram em 15 dias após o evento, e 106 mortes antes dos 15 dias.


## Prevendo se o paciente irá morrer

In [ ]:
#Modelo de regressão logística
from sklearn.linear_model import LogisticRegression

#Criando treino e teste
from sklearn.model_selection import train_test_split

#Avalição do modelo
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

In [ ]:
train, test = train_test_split(df_interest_data, train_size=0.7, stratify=df_interest_data['target'], random_state=2022)

In [ ]:
X = train['age'].values
X = np.array(X).reshape(-1, 1)
y = train['target'].values

In [ ]:
clf1 = LogisticRegression(random_state=2022).fit(X, y)

In [ ]:
predicts1 = clf1.predict(X)

In [ ]:
clf1.score(X, y)

0.71

In [ ]:
tn, fp, fn, tp = confusion_matrix(y, predicts1).ravel()
confusion_matrix(y, predicts1)

array([[106,  20],
       [ 38,  36]])

In [ ]:
(tn, fp, fn, tp)

(106, 20, 38, 36)

In [ ]:
balanced_accuracy_score(y, predicts1)

0.6638781638781639

## Outro classificador

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [ ]:
xgb_cl1 = xgb.XGBClassifier()

In [ ]:
xgb_cl1.fit(X, y)

XGBClassifier()

In [ ]:
preds1 = xgb_cl1.predict(X)

In [ ]:
accuracy_score(y, preds1)

0.795

In [ ]:
tn1, fp1, fn1, tp1 = confusion_matrix(y, preds1).ravel()
confusion_matrix(y, preds1)

array([[116,  10],
       [ 31,  43]])

In [ ]:
(tn1, fp1, fn1, tp1)

(116, 10, 31, 43)

In [ ]:
balanced_accuracy_score(y, preds1)

0.7508580008580008